<a href="https://colab.research.google.com/github/radhakrishnan-omotec/football-repo/blob/main/Part1_Data_Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Exploration
Before we get into building our xG model, we need to consider what sort of data we are interested in. Obviously, we need a large collection of shot data but more importantly we need the data to describe the type of shots that result in goals. We can deduce that the most important factors we need would be the distance from goal when the shot was taken, the angle with respect to the goal and what part of the body the shot was taken with.


In [2]:
!git clone https://github.com/radhakrishnan-omotec/football-repo.git

Cloning into 'football-repo'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 24 (delta 7), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (24/24), 940.73 KiB | 10.69 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [1]:
import pandas as pd
import numpy as np
import os
import json
import requests  # Added for downloading dataset
from urllib.parse import urlparse  # Added for parsing URL
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Arc

In [9]:
# Function to download dataset from URL and save it to the specified path
def download_dataset(url, save_path):
    response = requests.get(url)
    with open(save_path, 'wb') as file:
        file.write(response.content)

# Specify the URL and save path
dataset_url = "https://figshare.com/ndownloader/files/14464685"
save_path = "/content/football-repo/dataset/event_data/events.zip"


# Download the dataset
download_dataset(dataset_url, save_path)

# Add code to extract the contents if needed

# Specify the directory where the extracted files are stored
directory = '/content/football-repo/dataset/event_data'

# Create a list of the json files from the directory
jsonfiles = []
for path in os.listdir(directory):
    jsonfiles.append(os.path.join(directory, path))

print(jsonfiles)

['/content/football-repo/dataset/event_data/events.zip']


In [11]:
import zipfile

# Specify the directory where the extracted files should be stored
extracted_directory = '/content/football-repo/dataset/event_data/extracted'

# Specify the path to the downloaded zip file
zip_file_path = '/content/football-repo/dataset/event_data/events.zip'

# Create the directory if it doesn't exist
os.makedirs(extracted_directory, exist_ok=True)

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_directory)

# Create a list of the extracted json files
extracted_json_files = []
for path in os.listdir(extracted_directory):
    extracted_json_files.append(os.path.join(extracted_directory, path))

print("List of extracted JSON files:")
print(extracted_json_files)

List of extracted JSON files:
['/content/football-repo/dataset/event_data/extracted/events_World_Cup.json', '/content/football-repo/dataset/event_data/extracted/events_England.json', '/content/football-repo/dataset/event_data/extracted/events_European_Championship.json', '/content/football-repo/dataset/event_data/extracted/events_Spain.json', '/content/football-repo/dataset/event_data/extracted/events_France.json', '/content/football-repo/dataset/event_data/extracted/events_Germany.json', '/content/football-repo/dataset/event_data/extracted/events_Italy.json']
